In [1]:
import ee
ee.Initialize()

import geemap
import pandas as pd
import geopandas as gpd
import numpy as np 

import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

pd.options.display.float_format = '{:.3f}'.format

In [2]:
#pip install gspread oauth2client

#### area

In [3]:
aoi = ee.FeatureCollection('projects/ee-cocoacmr/assets/admin/cmr_admbnda_adm1')

In [4]:
# Compute area (ha) for each region in AOI
aoi_with_area = aoi.map(lambda f: f.set('Total_Area_ha', f.geometry().area().divide(10000)))

area_list = aoi_with_area.reduceColumns(
    reducer=ee.Reducer.toList(2),
    selectors=['ADM1_EN', 'Total_Area_ha']
).getInfo()['list']

area_dict = {name: area for name, area in area_list}

#### input maps

In [14]:
lc_image = ee.Image('projects/ee-cocoacmr/assets/land_cover/CMR_land_cover_2020')
tnt_image = ee.Image('projects/ee-cocoacmr/assets/land_cover/CMR_TNTMMU_2020')
ipcc_image = ee.Image('projects/ee-cocoacmr/assets/land_cover/CMR_IPCC_2020')
risk_image = ee.Image('projects/ee-cocoacmr/assets/outputs/CMR_EUDR_risk')
frag_image = ee.Image('projects/ee-cocoacmr/assets/land_cover/CMR_TNTMMU_frag_2020')

In [15]:
images = [lc_image, tnt_image, ipcc_image,risk_image,frag_image]

In [16]:
# resolution of the maps
scale = 10

In [24]:
# band name 
lc_band = 'LC_2020'
tnt_band = 'FNF_2020'
ipcc_band = 'IPCC_2020'
risk_band = 'risk'
frag_band = 'fragmentation'

In [25]:
band_names = [lc_band,tnt_band,ipcc_band,risk_band,frag_band]

In [26]:
# list of class codes
lc_labels = [11,12,13,14,15,21,22,30,41,42,50,60]
tnt_labels = [0,1]
ipcc_labels = [10,20,30,40,50,60]
risk_labels =[0,1,2,3,4,5,6,7,8,9,10]
frag_labels = [0,1,2,3,4]

In [27]:
class_label_sets = [
    [11,12,13,14,15,21,22,30,41,42,50,60],
    [0,1],
    [10,20,30,40,50,60],
    [0,1,2,3,4,5,6,7,8,9,10],
    [0,1,2,3,4]
]

In [28]:
labels = band_names
# Function to compute area per class
def compute_area(image, band, label, valid_classes, region, region_name):
    classified = image.select(band)
    pixel_area = ee.Image.pixelArea().clip(region.geometry())
    area_image = pixel_area.addBands(classified)

    stats = area_image.reduceRegion(
        reducer=ee.Reducer.sum().group(groupField=1, groupName='class'),
        geometry=region.geometry(),
        scale=30,
        maxPixels=1e12
    )

    groups = ee.List(stats.get('groups'))

    def format_group(group):
        group = ee.Dictionary(group)
        cls = ee.Number(group.get('class'))
        return ee.Algorithms.If(
            valid_classes.contains(cls),
            {
                'Label': label,
                'Region': region_name,
                'Band': band,
                'Class': cls,
                'Area (ha)': ee.Number(group.get('sum')).divide(10000)
            },
            None
        )

    return groups.map(format_group).removeAll([None])

# Collect all results
results = ee.List([])

regions = aoi.toList(aoi.size())
n_regions = regions.size().getInfo()

for i in range(n_regions):
    feature = ee.Feature(regions.get(i))
    region_dict = feature.toDictionary().getInfo()
    region_name = region_dict['ADM1_EN']


    for img, band, label, class_labels in zip(images, band_names, labels, class_label_sets):
        result = compute_area(img, band, label, ee.List(class_labels), feature, region_name)
        results = results.cat(result)

# Convert to DataFrame
df = pd.DataFrame(results.getInfo())

# Add band_class as column header
df['Band_Class'] = df['Band'] + '_' + df['Class'].astype(str)

In [29]:
df

,Area (ha),Band,Class,Label,Region,Band_Class
0,3703748.001,LC_2020,11,LC_2020,Adamawa,LC_2020_11
1,596713.934,LC_2020,12,LC_2020,Adamawa,LC_2020_12
2,130974.747,LC_2020,13,LC_2020,Adamawa,LC_2020_13
3,292360.085,LC_2020,15,LC_2020,Adamawa,LC_2020_15
4,6.742,LC_2020,21,LC_2020,Adamawa,LC_2020_21
...,...,...,...,...,...,...
322,32567.565,risk,10,risk,West,risk_10
323,6086.311,fragmentation,1,fragmentation,West,fragmentation_1
324,619059.246,fragmentation,2,fragmentation,West,fragmentation_2
325,105195.942,fragmentation,3,fragmentation,West,fragmentation_3


In [30]:
# Create a unique key per region
df['Region'] = df['Region'].astype(str)

# Create band_class column
df['Band_Class'] = df['Band'] + '_' + df['Class'].astype(str)

# Pivot table: one row per Region, one column per Band_Class
df_pivot = df.pivot_table(
    index='Region',
    columns='Band_Class',
    values='Area (ha)',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Optional: sort columns for consistency
df_pivot = df_pivot.sort_index(axis=1)

In [31]:
df_pivot

Band_Class,FNF_2020_0,FNF_2020_1,IPCC_2020_10,IPCC_2020_20,IPCC_2020_30,IPCC_2020_40,IPCC_2020_50,IPCC_2020_60,LC_2020_11,LC_2020_12,...,risk_1,risk_10,risk_2,risk_3,risk_4,risk_5,risk_6,risk_7,risk_8,risk_9
0,1651833.345,4747626.847,4752601.482,203540.114,1280163.525,51897.665,14391.326,96866.079,3703748.001,596713.934,...,1257116.750,435530.310,5780.248,60382.120,156602.425,38485.020,85958.480,2268.167,3500838.709,831526.640
1,1266593.307,5605575.596,5605851.499,62670.479,952152.705,42601.096,43191.599,165701.525,4176358.956,133094.981,...,821555.307,1195234.397,75211.788,328303.292,578244.661,278995.446,132983.132,16571.275,1653442.039,1774061.210
2,908168.546,10090974.401,10090802.436,24810.701,710001.377,39371.860,13837.688,120318.884,8990101.952,229948.073,...,754639.951,5932069.744,32282.827,205984.235,331538.647,148535.892,126071.060,17252.145,1589842.288,1827986.010
3,3412491.302,3578.666,3573.616,1443262.942,1442553.667,480436.695,31716.030,14527.019,3312.801,146.626,...,3060265.359,282961.738,0.000,0.000,49.429,0.000,1762.187,0.176,15952.965,3.609
4,341549.429,1679451.791,1672094.886,58322.180,186076.855,46705.410,28552.989,29248.900,1306835.896,18134.327,...,198290.585,448963.641,78700.441,83127.432,166606.009,32235.382,72187.791,6394.254,383154.423,518504.978
5,4312365.132,2303214.931,2298962.898,1481454.591,2673330.244,87338.793,21762.731,52730.807,1920304.891,296497.456,...,3602868.975,755012.696,0.000,6494.109,27739.222,4251.619,63539.447,18.799,1820799.606,288872.201
6,767834.686,973922.526,975214.729,147050.502,563597.874,13240.732,20677.866,21975.509,585274.115,129112.162,...,562367.997,123744.149,2793.617,31306.564,100316.882,29812.792,17056.921,373.772,744597.091,123309.167
7,149428.067,4567884.806,4542066.480,71997.949,67508.633,5224.925,11451.369,19063.517,4060073.059,24699.069,...,57393.602,1739731.385,222495.743,161596.106,150845.302,147422.705,115755.469,10107.605,345114.883,1762785.554
8,250621.878,2225802.165,2222583.265,66541.712,132436.333,24441.445,13837.821,16583.467,1780783.390,20376.739,...,111810.368,902101.864,90583.057,43761.534,140712.303,43429.252,50902.973,1054.218,412126.025,659536.402
9,660962.845,734471.739,736296.389,180779.174,396921.095,24067.854,39057.142,18312.930,447791.288,100586.819,...,539334.759,32567.565,1073.603,28634.823,89652.290,10819.886,21180.442,1582.576,590103.649,63609.715


In [32]:
df_pivot['Total_Area_ha'] = df_pivot['Region'].map(area_dict)

# Reorder: put Region first, then Total_Area_ha, then all other columns
cols = df_pivot.columns.tolist()
cols.remove('Region')
cols.remove('Total_Area_ha')
cols = ['Region', 'Total_Area_ha'] + cols

df_pivot = df_pivot[cols].reset_index(drop=True)

In [33]:
print(df_pivot.columns.tolist())

['Region', 'Total_Area_ha', 'FNF_2020_0', 'FNF_2020_1', 'IPCC_2020_10', 'IPCC_2020_20', 'IPCC_2020_30', 'IPCC_2020_40', 'IPCC_2020_50', 'IPCC_2020_60', 'LC_2020_11', 'LC_2020_12', 'LC_2020_13', 'LC_2020_14', 'LC_2020_15', 'LC_2020_21', 'LC_2020_22', 'LC_2020_30', 'LC_2020_41', 'LC_2020_42', 'LC_2020_50', 'LC_2020_60', 'fragmentation_1', 'fragmentation_2', 'fragmentation_3', 'fragmentation_4', 'risk_1', 'risk_10', 'risk_2', 'risk_3', 'risk_4', 'risk_5', 'risk_6', 'risk_7', 'risk_8', 'risk_9']


In [34]:
#rename columns
rename_dict = {
    'Total_Area_ha':'Total Area (ha)',
    'FNF_2020_0':'TNT Non-Forest', 
    'FNF_2020_1':'TNT Forest',
    'classification_11':'LC Dense Forest', 
    'classification_12':'LC Open Forest', 
    'classification_13':'LC Woodland Savanna', 
    'classification_14':'LC Mangrove', 
    'classification_15':'LC Secondary Forest', 
    'classification_21':'LC Forest Plantation', 
    'classification_22':'LC Agriculture', 
    'classification_30':'LC Grassland', 
    'classification_41':'LC Wetland', 
    'classification_42':'LC Water', 
    'classification_50':'LC Built-up', 
    'classification_60':'LC Bare', 
    'fragmentation_1':'Patch Forest', 
    'fragmentation_2':'Outer Edge', 
    'fragmentation_3':'Inner Edge', 
    'fragmentation_4':'Core Forest', 
    'remapped_10':'IPCC Forest', 
    'remapped_20':'IPCC Cropland', 
    'remapped_30':'IPCC Grassland', 
    'remapped_40':'IPCC Wetland', 
    'remapped_50':'IPCC Human Surface', 
    'remapped_60':'IPCC Other', 
    'risk_1':'Risk 1', 
    'risk_2':'Risk 2', 
    'risk_3':'Risk 3', 
    'risk_4':'Risk 4', 
    'risk_5':'Risk 5', 
    'risk_6':'Risk 6', 
    'risk_7':'Risk 7',
    'risk_8':'Risk 8', 
    'risk_9':'Risk 9',
    'risk_10':'Risk 10' 
}

df_pivot = df_pivot.rename(columns=rename_dict)
df_pivot

Band_Class,Region,Total Area (ha),TNT Non-Forest,TNT Forest,IPCC_2020_10,IPCC_2020_20,IPCC_2020_30,IPCC_2020_40,IPCC_2020_50,IPCC_2020_60,...,Risk 1,Risk 10,Risk 2,Risk 3,Risk 4,Risk 5,Risk 6,Risk 7,Risk 8,Risk 9
0,Adamawa,6426965.721,1651833.345,4747626.847,4752601.482,203540.114,1280163.525,51897.665,14391.326,96866.079,...,1257116.750,435530.310,5780.248,60382.120,156602.425,38485.020,85958.480,2268.167,3500838.709,831526.640
1,Centre,6902402.497,1266593.307,5605575.596,5605851.499,62670.479,952152.705,42601.096,43191.599,165701.525,...,821555.307,1195234.397,75211.788,328303.292,578244.661,278995.446,132983.132,16571.275,1653442.039,1774061.210
2,East,11047836.486,908168.546,10090974.401,10090802.436,24810.701,710001.377,39371.860,13837.688,120318.884,...,754639.951,5932069.744,32282.827,205984.235,331538.647,148535.892,126071.060,17252.145,1589842.288,1827986.010
3,Far-North,3429711.832,3412491.302,3578.666,3573.616,1443262.942,1442553.667,480436.695,31716.030,14527.019,...,3060265.359,282961.738,0.000,0.000,49.429,0.000,1762.187,0.176,15952.965,3.609
4,Littoral,2029928.126,341549.429,1679451.791,1672094.886,58322.180,186076.855,46705.410,28552.989,29248.900,...,198290.585,448963.641,78700.441,83127.432,166606.009,32235.382,72187.791,6394.254,383154.423,518504.978
5,North,6643340.431,4312365.132,2303214.931,2298962.898,1481454.591,2673330.244,87338.793,21762.731,52730.807,...,3602868.975,755012.696,0.000,6494.109,27739.222,4251.619,63539.447,18.799,1820799.606,288872.201
6,North-West,1749292.533,767834.686,973922.526,975214.729,147050.502,563597.874,13240.732,20677.866,21975.509,...,562367.997,123744.149,2793.617,31306.564,100316.882,29812.792,17056.921,373.772,744597.091,123309.167
7,South,4738349.909,149428.067,4567884.806,4542066.480,71997.949,67508.633,5224.925,11451.369,19063.517,...,57393.602,1739731.385,222495.743,161596.106,150845.302,147422.705,115755.469,10107.605,345114.883,1762785.554
8,South-West,2487270.441,250621.878,2225802.165,2222583.265,66541.712,132436.333,24441.445,13837.821,16583.467,...,111810.368,902101.864,90583.057,43761.534,140712.303,43429.252,50902.973,1054.218,412126.025,659536.402
9,West,1401530.364,660962.845,734471.739,736296.389,180779.174,396921.095,24067.854,39057.142,18312.930,...,539334.759,32567.565,1073.603,28634.823,89652.290,10819.886,21180.442,1582.576,590103.649,63609.715


In [35]:
df_pivot.to_csv('/home/sepal-user/cocoa-CMR/CMR_map_areas.csv',index=False)